# chinese graph

In [111]:
# coding: utf-8

import codecs
import networkx as nx
import numpy as np

In [128]:
f = codecs.open('./chinesegraph/chinese_words.csv', 'rb', encoding='utf-8')
words = [line.strip() for line in f]\
print 'number of words : ', len(words)

In [130]:
chars = set()
for w in words:
    chars = chars.union(set( w ))
    # Creating a little 'index'
print 'number of chars : ', len(chars)        

In [134]:
G = nx.Graph()
for ch in chars:
    G.add_node(ch,{'chinese':ch})

print 'number of chars nodes : ', G.number_of_nodes()

In [137]:
for i,w in enumerate(words):
    G.add_node(str(i),{'chinese': w })
    for ch in w:
        G.add_edge(str(i),ch)

print 'number of nodes (chars + words): ', G.number_of_nodes()

nx.write_graphml(G,'chinese_graph.graphml')

# word embedding

In [190]:
import networkx as nx
import numpy as np
import scipy as sp
import time
from random import choice
from math import log
import codecs

In [320]:
IS_CHINESE_TEST = True

if IS_CHINESE_TEST:
    G = nx.read_graphml('./data/graphs/chinese_graph.graphml', unicode) # Chinese graph
else:
    G = nx.read_graphml('graphs/bipartite_graph.graphml', unicode) # Korean language graph    

In [321]:
gen = nx.connected_component_subgraphs(G)
G = next(gen)
output_dir = "tmpDistancesFull2"

In [322]:
sts = nx.bipartite.sets(G)

In [323]:
words = None
hanjas = None
wordSet = None

if len(sts[0]) > 6000: 
    words = list(sts[0])
    wordSet = sts[0]
    hanjas = list(sts[1])
else: 
    words = list(sts[1])
    wordSet = sts[1]
    hanjas = list(sts[0])


In [324]:
wordic = {G.node[e]['chinese']:k for k,e in enumerate(words)}

In [325]:
synonyms = None

In [326]:
with codecs.open('./data/synonyms/all_chinese_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]

In [327]:
synDP = []
randDP = []


In [328]:
A = nx.bipartite.biadjacency_matrix(G, hanjas)
A = A.astype(float)

In [329]:
weight = False # In case we chose not to do TF-IDF weighting

In [330]:
minK = 666
maxK = 668

In [331]:
U,s,V = sp.sparse.linalg.svds(A,800)
Vtr = V.transpose()

In [316]:
for pr in synonyms:

    word = pr.split(',')
    c1 = word[0]
    c2 = word[1]
    
    if c1 not in wordic or c2 not in wordic: continue

    v1 = Vtr[wordic[c1]]
    v2 = Vtr[wordic[c2]]
    synDP.append([ np.dot(v1,v2), c1.encode('utf-8'), c2.encode('utf-8') ] )

In [317]:
for _ in synonyms:
    v1 = choice(Vtr)
    v2 = choice(Vtr)
    randDP.append(np.dot(v1,v2))

In [318]:
output_dir = './output'
with codecs.open("{}/randdist_k={}.csv".format(output_dir,k), 'w', encoding='utf-8') as f:
    wr = csv.writer(f)
    wr.writerows([[r] for r in randDP])

In [319]:
with codecs.open("{}/syndist_k={}.csv".format(output_dir,k), 'w') as f:
    wr = csv.writer(f)
    wr.writerows(synDP)